In [1]:
import math
import numpy
import random
import pickle
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras import models, Model
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import matplotlib as mpl
from matplotlib import cm
from matplotlib.animation import FuncAnimation
from matplotlib.colors import ListedColormap
import multiprocessing
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
from tensorflow.keras import models, Model
import numpy as np

xl=[]
with open('papercon/smallgaussian.txt') as f:
    for line in f.readlines():
        line=[np.float64(line.strip('\n').split()[i])/(1e+10) for i in range(len(line.strip('\n').split()))]
        xl.append(line)

2023-08-13 07:08:35.998074: I tensorflow/core/platform/cpu_feature_guard.cc:183] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
from numba import guvectorize,cuda
def get_mean_vector(h_set):
    """
    :param h_set: list of vectors h, each vector is output of LSTM layer at a timestep
    :return: mean vector hbar
    """
    hbar = h_set[0]
    for i in range(1, len(h_set)):
        hbar += h_set[i]
    hbar = hbar / len(h_set)
    return hbar

def get_magnitude(vector):
    """
    :param vector: 1D numpy array
    :return: magnitude of vector
    """
    magnitude = 0
    for element in vector:
        magnitude += element ** 2
    return math.sqrt(magnitude)

def get_norm(vector):
    """
    :param vector: vector to normalise
    :return: norm of vector
    """
    return vector / get_magnitude(vector)

def project(vector, basis):
    """
    :param vector: vector to project onto basis
    :param basis: basis for poincare map
    :return: vector projected onto basis (dot product)
    """
    return vector.dot(basis)

def get_poincare_mapping(lstm, start, num_steps, intermediate_inputs=None):
    """
    get poincare mapping (projections at h_t, projections at h_{t+1})
    :param lstm: trained LSTM_layer
    :param start: starting input
    :param num_steps: number of iterations to perform, length of intermediate_inputs has to be num_steps - 1
    :param intermediate_inputs: list of x_t to input at each timestep, zero vectors if None, each vector has to be length start
    :return: poincare mapping
    """
    if intermediate_inputs is None:
        intermediate_inputs = [np.zeros(len(start), dtype=np.float64) for _ in range(num_steps - 1)]

    # get h_t at each timestep
    h_t = [lstm.step(start)[-1]]
    h_t_1 = [] # h_{t+1}
    for i in range(num_steps - 1):
        curr_h = lstm.step(intermediate_inputs[i])[-1]
        h_t.append(curr_h)
        h_t_1.append(curr_h)

    h_t.pop() # remove last element so h_t and h_{t+1} aligns
    return h_t, h_t_1

def main(n):       
    # numbers setup
    class LSTM_layer():
        @staticmethod
        def sigmoid(x):
            return 1 / (1 + np.exp(-x))

        @staticmethod
        def tanh(x): # for consistency
            return np.tanh(x)
        

        def __init__(self, weights):
            """
            :param weights: weights of LSTM layer
            """
            # transposing matrices for dot product
            self.W, self.U, self.b = np.transpose(weights[0]), np.transpose(weights[1]), np.transpose(weights[2])
            self.num_units = int(self.U.shape[1])
            self.split_weights()
            # LSTM trained stateless, initial C and h are zero vectors
            self.C = np.zeros((self.num_units), dtype=np.float64)
            self.h = np.zeros((self.num_units), dtype=np.float64)

        def split_weights(self):
            # weights are stored as (neuron_num, (i, f, c, o))
            self.W_i = np.ascontiguousarray(self.W[:self.num_units, :])
            self.W_f = np.ascontiguousarray(self.W[self.num_units:self.num_units * 2, :])
            self.W_c = np.ascontiguousarray(self.W[self.num_units * 2:self.num_units * 3, :])
            self.W_o = np.ascontiguousarray(self.W[self.num_units * 3:, :])

            self.U_i = np.ascontiguousarray(self.U[:self.num_units, :])
            self.U_f = np.ascontiguousarray(self.U[self.num_units:self.num_units * 2, :])
            self.U_c = np.ascontiguousarray(self.U[self.num_units * 2:self.num_units * 3, :])
            self.U_o = np.ascontiguousarray(self.U[self.num_units * 3:, :])

            self.b_i = np.ascontiguousarray(self.b[:self.num_units])
            self.b_f = np.ascontiguousarray(self.b[self.num_units:self.num_units * 2])
            self.b_c = np.ascontiguousarray(self.b[self.num_units * 2:self.num_units * 3])
            self.b_o = np.ascontiguousarray(self.b[self.num_units * 3:])

        def step(self, x_t):
            """
            Performs a timestep (propagating new input through layer)
            :return: array of activations [ft, it, cc, cc_update, c_out, ot, ht]
            """
            activations = []
            # forget step
            ft = self.get_ft(x_t)
            activations.append(ft)
            self.forget(ft)

            # "remembering" step
            it = self.get_it(x_t)
            activations.append(it)
            cc = self.get_CC(x_t)
            activations.append(cc)
            cc_update = self.get_CC_update(it, cc)
            activations.append(cc_update)
            self.remember(cc_update)

            # output step
            c_out = self.get_C_output()
            activations.append(c_out)
            ot = self.get_ot(x_t)
            activations.append(ot)
            output = self.output(c_out, ot)
            activations.append(output)

            return activations

        def reset(self):
            # call when done with one input (with all timesteps completed)
            # resets internal cell state and starting hidden state
            self.C = np.zeros((self.num_units), dtype=np.float64)
            self.h = np.zeros((self.num_units), dtype=np.float64)


        # vectorized activation propagation
        @staticmethod
        @guvectorize(
            ["float64[:, :], float64[:, :], float64[:], float64[:], float64[:], float64[:]"],
            "(n, m),(n, n),(m),(n),(n)->(n)",
            nopython=True
        )
        def get_ft_vec(W_f, U_f, x_t, h, b_f, res):
            wfx = W_f.dot(x_t)
            ufh = U_f.dot(h)
            sum_int = wfx + ufh
            sum_f = sum_int + b_f
            res[:] = 1 / (1 + np.exp(-sum_f))

        @staticmethod
        @guvectorize(
            ["float64[:, :], float64[:, :], float64[:], float64[:], float64[:], float64[:]"],
            "(n, m),(n, n),(m),(n),(n)->(n)",
            nopython=True
        )
        def get_it_vec(W_i, U_i, x_t, h, b_i, res):
            wix = W_i.dot(x_t)
            uih = U_i.dot(h)
            sum_int = wix + uih
            sum_f = sum_int + b_i
            res[:] = 1 / (1 + np.exp(-sum_f))

        @staticmethod
        @guvectorize(
            ["float64[:, :], float64[:, :], float64[:], float64[:], float64[:], float64[:]"],
            "(n, m),(n, n),(m),(n),(n)->(n)",
            nopython=True
        )
        def get_CC_vec(W_c, U_c, x_t, h, b_c, res):
            wcx = W_c.dot(x_t)
            uch = U_c.dot(h)
            sum_int = wcx + uch
            sum_f = sum_int + b_c
            res[:] = np.tanh(sum_f)

        @staticmethod
        @guvectorize(
            ["float64[:, :], float64[:, :], float64[:], float64[:], float64[:], float64[:]"],
            "(n, m),(n, n),(m),(n),(n)->(n)",
            nopython=True
        )
        def get_ot_vec(W_o, U_o, x_t, h, b_o, res):
            wox = W_o.dot(x_t)
            uoh = U_o.dot(h)
            sum_int = wox + uoh
            sum_f = sum_int + b_o
            res[:] = 1 / (1 + np.exp(-sum_f))

        # activations start
        # tanh activations don't see an improvement from vectorization (probably because tanh is already vectorized)
        def get_ft(self, x_t):
            # sigmoid(W_f . x_t + U_f . h_(t-1) + b_f) . is dot product
            # wfx = self.W_f.dot(x_t)
            # ufh = self.U_f.dot(self.h)
            # return LSTM_layer.sigmoid(wfx + ufh + self.b_f)
            return LSTM_layer.get_ft_vec(self.W_f, self.U_f, x_t, self.h, self.b_f)

        def get_it(self, x_t):
            # sigmoid(W_i . x_t + U_i . h_(t-1) + b_i)
            # wix = self.W_i.dot(x_t)
            # uih = self.U_i.dot(self.h)
            # return LSTM_layer.sigmoid(wix + uih + self.b_i)
            return LSTM_layer.get_it_vec(self.W_i, self.U_i, x_t, self.h, self.b_i)

        def get_CC(self, x_t):
            # candidate cell state before proportion
            # tanh(W_c . x_t + U_c . h_(t-1) + b_c)
            wcx = self.W_c.dot(x_t)
            uch = self.U_c.dot(self.h)
            return LSTM_layer.tanh(wcx + uch + self.b_c)
            # return LSTM_layer.get_CC_vec(self.W_c, self.U_c, x_t, self.h, self.b_c)

        def get_ot(self, x_t):
            # sigmoid(W_o . x_t + U_o . h_(t-1) + b_o)
            # wox = self.W_o.dot(x_t)
            # uoh = self.U_o.dot(self.h)
            # return LSTM_layer.sigmoid(wox + uoh + self.b_o)
            return LSTM_layer.get_ot_vec(self.W_o, self.U_o, x_t, self.h, self.b_o)

        def get_C_output(self):
            # cell state output before proportion
            # tanh(C_t)
            return LSTM_layer.tanh(self.C)

        def get_CC_update(self, it, cc):
            # candidate cell state after proportion, for updating cell state
            # it * cc, * is Hadamard product
            return it * cc
        # activations end


        # state updates start
        def forget(self, ft):
            # update old cell state in the forget step
            self.C = self.C * ft

        def remember(self, cc_update):
            # update old cell state with new information
            self.C = self.C + cc_update

        def output(self, c_output, ot):
            # proportionate the cell output vector for new output and hidden state
            self.h = c_output * ot
            return self.h
    
    num_timesteps = 500
    len_sequence = 75000
    start_count = 0 #99500
    end_point = 75000
    num_cells = 60
    
    
    lstm_made=xl[0:500]
    lstm_in=[lstm_made]
    lstm_in_gau=[]
    for i in range(500):
        lstm_in_gau.append(xl[i])
    lstm_in_gau=[lstm_in_gau]
        #x_in=np.zeros((1,500),int).tolist()
        # print(length[i])
        # if length[i] < 350:
        #     continue
        #x_in_gau = x[i].reshape((1, num_timesteps))+ random.gauss(0,1e-15)
        #x_in_gau = x_in + random.gauss(0,0.0001)
        #print(x_in[0])
        #print(x_in_gau)
    #print(np.shape(lstm_in))
        #print(lstm_in.dtype)
        #for i in range(0,500):
            #lstm_in[0][i]=np.float64(np.zeros((32,)))
        #lstm_in_late[0][0][0]=0.01
        #print(lstm_in_late)
        #for i in range(0,500):
            #lstm_in_gau[0][i]=np.float64(np.zeros((32,)))
        #print(lstm_in_gau.dtype,lstm_in_gau)
        #lstm_in_late_gau[0][0][0]=0.01
    lstm_in_gau[0][0]=lstm_in_gau[0][0]+np.float64(random.gauss(0,1e-10))
    a=[]
    path='unitestdense/weight.'+str(n+1)+'.txt'
    with open(path) as f:
            for line in f.readlines():
                    line=np.float64(line.strip('\n').split())
                    a.append(line)
    path11='unitestdense/weight_re.'+str(n+1)+'.txt'
    b=[]
    with open(path11) as f:
            for line in f.readlines():
                    line=np.float64(line.strip('\n').split())
                    b.append(line)
    a=[a]+[b]
    a.append(np.zeros((240,)))
    lstm = LSTM_layer(a)
    start = lstm_in[0][0]
    #print(np.array(start).shape)
    start_gau=lstm_in_gau[0][0]
        #print(lstm_in_late)
        #print(start,start_gau)
    print(start==start_gau)
    intermediate_steps=xl[500:]
    
        

        
    h_t_late, h_t_1_late = get_poincare_mapping(lstm, start, len_sequence, intermediate_steps)
    h_t_late.append(h_t_1_late[-1])
    hbar_late = get_mean_vector(h_t_late)
        
    h_t_late_gau, h_t_1_late_gau = get_poincare_mapping(lstm, start_gau, len_sequence, intermediate_steps)
        #print(h_t_late_gau[0])
    h_t_late_gau.append(h_t_1_late_gau[-1])
    late_set=[]

        #print(len(h_t_late_gau))
        #print(len(h_t_late))
    for j in range(len(h_t_late)-10, len(h_t_late)):

            vec1=np.array(h_t_late[j])
            vec2=np.array(h_t_late_gau[j])
            dist = np.float64(numpy.linalg.norm(vec1 - vec2))
            #if dist<=np.finfo(np.float32).eps:
                #dist=np.float32(0)
            dist_late=np.float64(numpy.log(dist+numpy.exp(-25)))
            
            late_set.append(dist_late)
    print(n,np.mean(late_set))   
    return np.mean(late_set)
        #if np.mean(late)<=np.float32(np.log(np.finfo(np.float32).eps)):
            #me=np.float32(-25)
        
         
        #plt.figure(figsize=(15,8)) 
        #plt.scatter(np.arange(len(late)),late,c='blue')
        #plt.scatter(h_t_late_gau[0:-2],h_t_late_gau[1:-1],c='grey')
        #plt.plot(h_t_late[-2],h_t_late[-1],'d',c='yellow')
        #plt.plot(h_t_late_gau[-2],h_t_late[-1],'s',c='orange')
        #plt.ylim(-30,2,32)
        #plt.xlabel('len_sequence')
        #plt.ylabel('log distance between ht` and ht')
        #plt.plot(h_t_late_gau[-2],h_t_late_gau[-1],'s',c='green')
        #plt.legend()
        #plt.savefig('LSTM70/str(n)_multiplier.png')
        #plt.title('distance of the optimal epoch')
        #a=str(n)+''+'opt'+'-'+'times'+'-'+'multilier'
        #plt.show()
        
        
if __name__ == "__main__":
    #pool=multipressing.Pool(processes=8)
    #for e in range(200,460):
    x=[]
    for i in range(1009):
        x.append(main(i))
    


[False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False]
0 -25.0
[False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False]
1 -25.0


In [5]:
with open('freezedense/dis.pkl', 'wb') as f:
    pickle.dump(x, f)

In [6]:
with open('freezedense/dis.pkl', 'rb') as f:
    y = pickle.load(f)

In [7]:
y

[-25.0,
 -25.0,
 -25.0,
 -25.0,
 -25.0,
 -1.0243492502003748,
 -0.889154532630888,
 -0.2623644248995247,
 0.23123770957059406,
 0.2552258594951918,
 0.15133484498065397,
 0.4018188167178975,
 0.48517596325362167,
 -0.7025050928919205,
 -25.0,
 0.4016414960490656,
 0.3691985815056551,
 -0.1918967687587222,
 0.5116363399769485,
 0.5616958822360585,
 0.5391516561698484,
 0.18180576574964125,
 0.33936971479515987,
 -0.7151738755396865,
 0.4703502609676982,
 0.9616275093899576,
 -1.9259900131524332,
 0.9417207862641043,
 0.8192903163028195,
 0.35668757086226655,
 -0.09571878565773533,
 0.6040947645745125,
 0.9490775301588762,
 0.8974293078924394,
 0.9467075153983666,
 -0.9971855502275672,
 0.27541194943582,
 0.8094924893699481,
 0.05203952010973241,
 -0.2592619917154644,
 0.8224587292085696,
 -0.11890892614710924,
 0.5023091996338589,
 0.36129446533576304,
 0.055228367375607726,
 0.7372637081823382,
 -0.2860903289415241,
 -0.5140616592134807,
 0.80481791463976,
 0.1976286393440178,
 1.00404